In [ ]:
import numpy as np
from PIL import Image
import paddle
import passl
from passl.models import vision_transformer
from passl.models.utils import tome
from passl.data import preprocess as transforms

In [ ]:
# Download models and assets
!mkdir -p models
if not os.path.exists('models/imagenet2012-ViT-B_16-224.pdparams'):
    !wget -O ./models/imagenet2012-ViT-B_16-224.pdparams https://passl.bj.bcebos.com/models/vit/v2.4/imagenet2012-ViT-B_16-224.pdparams

!mkdir -p images
if not os.path.exists('images/husky.png'):
    !wget -O images/husky.png https://passl.bj.bcebos.com/dataset/test_images/husky.png

In [ ]:
model = vision_transformer.ViT_base_patch16_224()
model.load_pretrained('models/imagenet2012-ViT-B_16-224')

In [ ]:
transform_val = transforms.Compose([
    transforms.Resize(
        size=256, interpolation="bicubic", backend="pil"),  # 3 is bicubic
    transforms.CenterCrop(size=224),
    transforms.NormalizeImage(
        scale=1.0 / 255.0,
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5],
        order='hwc'),
    transforms.ToCHWImage()
])

img = Image.open("images/husky.png")
img = transform_val(img)[None, ...]
img = paddle.to_tensor(img)

In [ ]:
tome.apply_patch(model)

model.r = 8
#model.r = 16 #[25] * 8  # 8 / 24 layers

logits = model(img)
preds = paddle.nn.functional.softmax(logits)
out = preds.argsort(descending=True)[..., :5]
print(out)